In [1]:
import pandas as pd
from pandasql import sqldf # type: ignore

# Set up pandas and pandasql
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pysqldf = lambda q: sqldf(q, globals())

# Load the data
subscriptions = pd.read_csv("subscriptions.csv").rename(columns={"id": "subscription_id_pk"})
universities = pd.read_csv("universities.csv").rename(columns={"id": "university_id_pk", "name": "university_name"})
users = pd.read_csv("users.csv").rename(columns={"id": "user_id_pk", "name": "user_name"})

In [2]:
q = """
select
    case
        when CAST(strftime('%j', created_at) AS INTEGER) >= 304 then strftime('%Y', created_at) || "-" || strftime('%Y', date(created_at, '+1 year'))
        else strftime('%Y', date(created_at, '-1 year')) || "-" || strftime('%Y', created_at)
    end as user_created_at_school_year,
    case
        when CAST(strftime('%j', term_start) AS INTEGER) >= 304 then strftime('%Y', term_start) || "-" || strftime('%Y', date(term_start, '+1 year'))
        else strftime('%Y', date(term_start, '-1 year')) || "-" || strftime('%Y', term_start)
    end as term_start_school_year,
    case
        when CAST(strftime('%j', term_end) AS INTEGER) >= 304 then strftime('%Y', term_end) || "-" || strftime('%Y', date(term_end, '+1 year'))
        else strftime('%Y', date(term_end, '-1 year')) || "-" || strftime('%Y', term_end)
    end as term_end_school_year,
    julianday(date(term_end)) - julianday(date(term_start)) as term_length,
    *
from users
join universities on users.university_id = universities.university_id_pk
left join (select *, row_number() over (partition by user_id order by term_start asc) as payment_number from subscriptions) as subs on users.user_id_pk = subs.user_id
"""

joined_df = pysqldf(q)

### Top 75 schools of paid subscribers

Table sorted by the sum of total_students and n_paid_students descending

In [3]:
q = """
select
    university_id_pk,
    university_name,
    count(user_id_pk) as total_students,
    sum(case when transaction_type = "PAID" then 1 else 0 end) as n_paid_subs,
    ROUND(100 * SUM(CASE WHEN transaction_type = "PAID" THEN 1 ELSE 0 END) / CAST(COUNT(user_id_pk) AS FLOAT), 2) conversion_rate
from joined_df
where (payment_number = 1 or payment_number is null)
and country = "United States of America"
and university_name != "Other"
group by 1,2
order by total_students + n_paid_subs desc
limit 75
"""

pysqldf(q)

,university_id_pk,university_name,total_students,n_paid_subs,conversion_rate
0,30,Touro College of Osteopathic Medicine,3064,1313,42.85
1,10,Lake Erie College of Osteopathic Medicine,2363,1127,47.69
2,34,Philadelphia College of Osteopathic Medicine,2479,920,37.11
3,15,Midwestern University Chicago College of Osteo...,2366,404,17.08
4,8,Western University of Health Sciences College ...,2264,503,22.22
5,391,Texas Tech University Health Sciences Center S...,1784,955,53.53
6,13,Des Moines University College of Osteopathic M...,1835,510,27.79
7,51,University of Arkansas for Medical Sciences Co...,1598,724,45.31
8,193,Saint Louis University School of Medicine,1295,976,75.37
9,203,Wake Forest School of Medicine of Wake Forest ...,1474,755,51.22


In [4]:
q = """
with paid_subs as (
    select
        university_id_pk,
        university_name,
        sum(case when transaction_type = "PAID" then 1 else 0 end) as n_students,
        "paid" as payment_status
    from joined_df
    where (payment_number = 1 or payment_number is null)
    and country = "United States of America"
    and university_name != "Other"
    group by 1,2
    order by 3 desc
    limit 75
),
total_subs as (
    select
        university_id_pk,
        university_name,
        count(user_id_pk) as n_students,
        "total" as payment_status
    from joined_df
    where (payment_number = 1 or payment_number is null)
    and country = "United States of America"
    and university_name != "Other"
    and university_id_pk in (select university_id_pk from paid_subs)
    group by 1,2
)

select * from paid_subs
union
select * from total_subs
"""

sub_breakdown = pysqldf(q)

In [5]:
import plotly.express as px # type: ignore
fig = px.histogram(
    sub_breakdown, 
    x="university_name", 
    y="n_students",
    color='payment_status', 
    barmode='group',
    height=600
)
fig.update_xaxes(
    visible=False,
    tickmode='array', 
    ticktext=[x for x in sub_breakdown['university_name'].tolist()],
    tickvals=sub_breakdown['university_name'].tolist(),
    categoryorder='total descending'
)
fig.update_yaxes(title_text="Number of Students")
fig.update_layout(
    title="Paid Subscribers vs. Total Students by University",
    legend_title="Subscription Status",
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.80
    )
)
fig.show()